In [1]:
import logging
from typing import Annotated

from autogen import register_function
from autogen.agentchat import ConversableAgent
from fixtures import openai_gpt4o_mini_llm_config

from fastagency.core import Chatable, ConsoleIO, IOMessage
from fastagency.core.autogen.base import AutoGenWorkflows

In [2]:
# Get the logger
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

logger.handlers = []

# Create a stream handler
handler = logging.StreamHandler()
handler.setLevel(logging.DEBUG)

# Create a formatter and set it for the handler
formatter = logging.Formatter("[%(levelname)s] %(message)s")
handler.setFormatter(formatter)

# Add the handler to the logger
logger.addHandler(handler)

# Log messages
logger.warning("warning message")
logger.info("info message")

[WARNING] warning message
[INFO] info message


In [3]:
wf = AutoGenWorkflows()


@wf.register(name="simple_learning", description="Student and teacher learning chat")
def simple_workflow(io: Chatable, initial_message: str, session_id: str) -> str:
    student_agent = ConversableAgent(
        name="Student_Agent",
        system_message="You are a student willing to learn.",
        llm_config=openai_gpt4o_mini_llm_config,
    )
    teacher_agent = ConversableAgent(
        name="Teacher_Agent",
        system_message="You are a math teacher.",
        llm_config=openai_gpt4o_mini_llm_config,
    )

    chat_result = student_agent.initiate_chat(
        teacher_agent,
        message=initial_message,
        summary_method="reflection_with_llm",
        max_turns=5,
    )

    return chat_result.summary

In [4]:
initial_message="What is triangle inequality?"

io = ConsoleIO()

io.process_message(IOMessage(sender="user", recepient="workflow", heading="Workflow BEGIN", content={"body": f"Starting workflow with initial_message: {initial_message}"}))

result = wf.run(name="simple_learning", session_id="session_id", io=io.create_subconversation(), initial_message=initial_message)

io.process_message(IOMessage(sender="user", recepient="workflow", heading="Workflow END", content={"body": f"Ending workflow with result: {result}"}))

[ERROR] End of message: self.current_message={'sender': 'Student_Agent', 'recepient': 'Teacher_Agent', 'content': {'body': 'What is triangle inequality?\n'}}
[ERROR] End of message: self.current_message={'heading': 'AUTO REPLY', 'sender': 'Teacher_Agent', 'recepient': 'Student_Agent', 'content': {'body': 'The triangle inequality is a fundamental property of triangles in geometry that states that, for any triangle, the sum of the lengths of any two sides must be greater than the length of the third side. This can be expressed mathematically as follows:\n\nIf a triangle has sides of lengths \\( a \\), \\( b \\), and \\( c \\), then the triangle inequality can be stated as:\n\n1. \\( a + b > c \\)\n2. \\( a + c > b \\)\n3. \\( b + c > a \\)\n\nThis means that no one side can be equal to or greater than the sum of the other two sides. The triangle inequality is essential for determining whether three lengths can form a triangle and is also a foundational concept in various areas of mathema

+--------------------------------------------------------------------------------+
|
| user -> workflow: Workflow BEGIN
|
| Starting workflow with initial_message: What is triangle inequality?
+--------------------------------------------------------------------------------+

    +--------------------------------------------------------------------------------+
    |
    | Student_Agent -> Teacher_Agent: 
    |
    | What is triangle inequality?

    +--------------------------------------------------------------------------------+

    +--------------------------------------------------------------------------------+
    |
    | Teacher_Agent -> Student_Agent: AUTO REPLY
    |
    | The triangle inequality is a fundamental property of triangles in 
    | geometry that states that, for any triangle, the sum of the lengths of
    |  any two sides must be greater than the length of the third side. This
    |  can be expressed mathematically as follows:
    | 
    | If a triangle has side

In [5]:
# wf = AutoGenWorkflows()

@wf.register(name="exam_practice", description="Student and teacher chat")
def exam_learning(io: Chatable, initial_message: str, session_id: str) -> str:

    def is_termination_msg(msg: str) -> bool:
        return msg["content"] is not None and "TERMINATE" in msg["content"]

    student_agent = ConversableAgent(
        name="Student_Agent",
        system_message="You are a student writing a practice test. Your task is as follows:\n"
            "  1) Retrieve exam questions by calling a function.\n"
            "  2) Write a draft of proposed answers and engage in dialogue with your tutor.\n"
            "  3) Once you are done with the dialogue, register the final answers by calling a function.\n"
            "  4) Retrieve the final grade by calling a function.\n"
            "Finally, terminate the chat by saying 'TERMINATE'.",
        llm_config=openai_gpt4o_mini_llm_config,
        human_input_mode="NEVER",
        is_termination_msg=is_termination_msg,
    )
    teacher_agent = ConversableAgent(
        name="Teacher_Agent",
        system_message="You are a teacher.",
        llm_config=openai_gpt4o_mini_llm_config,
        human_input_mode="NEVER",
        is_termination_msg=is_termination_msg,
    )

    def retrieve_exam_questions(message: Annotated[str, "Message for examiner"]) -> str:
        # return "1) Mona Lisa 2) Innovations 3) Florence at the time of Leonardo 4) The Last Supper 5) Vit"
        try:
            msg = IOMessage(
                sender="student",
                recepient="teacher",
                heading="Question for examiner",
                type="text_input",
                content={
                    "prompt": message,
                    "suggestions": ["1) Mona Lisa", "2) Innovations", "3) Florence at the time of Leonardo", "4) The Last Supper", "5) Vit"]
                },
            )
            return io.process_message(msg)
        except Exception as e:
            logger.error(f"retrieve_exam_questions() FAILED! {e}")
            return f"retrieve_exam_questions() FAILED! {e}"

    def write_final_answers(message: Annotated[str, "Message for examiner"]) -> str:
        try:
            msg = IOMessage(
                sender="function call logger",
                recepient="system",
                heading="Storing final answers",
                type="text_message",
                content={"body": message}
            )
            io.process_message(msg)
            return "Final answers stored."
        except Exception as e:
            logger.error(f"write_final_answers() FAILED! {e}")
            return f"write_final_answers() FAILED! {e}"

    def get_final_grade(message: Annotated[str, "Message for examiner"]) -> str:
        try:
            msg = IOMessage(
                    sender="student",
                    recepient="teacher",
                    heading="Final grade",
                    type="multiple_choice",
                    content={"prompt": message, "choices": ["A", "B", "C", "D", "F"]},
            )
            return io.process_message(msg)
        except Exception as e:
            logger.error(f"get_final_grade() FAILED! {e}")
            return f"get_final_grade() FAILED! {e}"

    register_function(
        retrieve_exam_questions,
        caller=student_agent,
        executor=teacher_agent,
        name="retrieve_exam_questions",
        description="Get exam questions from examiner",
    )

    register_function(
        write_final_answers,
        caller=student_agent,
        executor=teacher_agent,
        name="write_final_answers",
        description="Write a final answers to exam questions to examiner, but only after discussing with the tutor first.",
    )

    register_function(
        get_final_grade,
        caller=student_agent,
        executor=teacher_agent,
        name="get_final_grade",
        description="Get the final grade after submitting the answers.",
    )

    chat_result = teacher_agent.initiate_chat(
        student_agent,
        message=initial_message,
        summary_method="reflection_with_llm",
        max_turns=10,
    )

    return chat_result.summary

In [6]:
initial_message="Prepare for the test about Leonardo da Vinci."

io = ConsoleIO()

io.process_message(IOMessage(sender="user", recepient="workflow", heading="Workflow BEGIN", content={"body": f"Starting workflow with initial_message: {initial_message}"}))

result = wf.run(name="exam_practice", session_id="session_id", io=io.create_subconversation(), initial_message=initial_message)

io.process_message(IOMessage(sender="user", recepient="workflow", heading="Workflow END", content={"body": f"Ending workflow with result: {initial_message}"}))

+--------------------------------------------------------------------------------+
|
| user -> workflow: Workflow BEGIN
|
| Starting workflow with initial_message: Prepare for the test about 
| Leonardo da Vinci.
+--------------------------------------------------------------------------------+



[ERROR] End of message: self.current_message={'sender': 'Teacher_Agent', 'recepient': 'Student_Agent', 'content': {'body': 'Prepare for the test about Leonardo da Vinci.\n'}}
[ERROR] End of message: self.current_message={'sender': 'Student_Agent', 'recepient': 'Teacher_Agent', 'type': 'suggested_function_call', 'content': {'function_name': 'retrieve_exam_questions', 'call_id': 'call_WtPX2rq3saIIqk4hM4jjAhNX', 'arguments': {'message': 'Please provide the exam questions for the test about Leonardo da Vinci.'}}}


    +--------------------------------------------------------------------------------+
    |
    | Teacher_Agent -> Student_Agent: 
    |
    | Prepare for the test about Leonardo da Vinci.

    +--------------------------------------------------------------------------------+

    +--------------------------------------------------------------------------------+
    |
    | Student_Agent -> Teacher_Agent: Message type: suggested_function_call
    |
    | {
    |   "function_name": "retrieve_exam_questions",
    |   "call_id": 
    | "call_WtPX2rq3saIIqk4hM4jjAhNX",
    |   "arguments": {
    |     "message": 
    | "Please provide the exam questions for the test about Leonardo da 
    | Vinci."
    |   }
    | }
    +--------------------------------------------------------------------------------+



[ERROR] End of message: self.current_message={'type': 'function_call_execution', 'heading': 'RESPONSE FROM CALLING TOOL (call_WtPX2rq3saIIqk4hM4jjAhNX)', 'content': {'function_name': 'retrieve_exam_questions', 'call_id': 'call_WtPX2rq3saIIqk4hM4jjAhNX', 'retval': '1) Vitruvian man 2) Mona Lisa\n'}, 'sender': 'Teacher_Agent', 'recepient': 'Student_Agent'}


    +--------------------------------------------------------------------------------+
    |
    | Teacher_Agent -> Student_Agent: Message type: function_call_execution
    |
    | {
    |   "function_name": "retrieve_exam_questions",
    |   "call_id": 
    | "call_WtPX2rq3saIIqk4hM4jjAhNX",
    |   "retval": "1) Vitruvian man 2) Mona
    |  Lisa\n"
    | }
    +--------------------------------------------------------------------------------+



...

[ERROR] End of message: self.current_message={'sender': 'Teacher_Agent', 'recepient': 'Student_Agent', 'content': {'body': "Your final answer for the Mona Lisa is excellent! It is thorough, well-structured, and incorporates key enhancements that provide a deeper understanding of the painting's significance. \n\nIf you feel confident about this answer and are satisfied with the content, you can go ahead and register it for submission. It effectively covers all essential aspects, making it a strong response to the exam question.\n\nIf you have any other questions or need further assistance, feel free to ask! Otherwise, good luck with your test!\n"}}


    +--------------------------------------------------------------------------------+
    |
    | Teacher_Agent -> Student_Agent: 
    |
    | Your final answer for the Mona Lisa is excellent! It is thorough, 
    | well-structured, and incorporates key enhancements that provide a 
    | deeper understanding of the painting's significance. 
    | 
    | If you feel 
    | confident about this answer and are satisfied with the content, you 
    | can go ahead and register it for submission. It effectively covers all
    |  essential aspects, making it a strong response to the exam question.
    | 
    | 
    | If you have any other questions or need further assistance, feel 
    | free to ask! Otherwise, good luck with your test!

    +--------------------------------------------------------------------------------+



[ERROR] End of message: self.current_message={'sender': 'Student_Agent', 'recepient': 'Teacher_Agent', 'type': 'suggested_function_call', 'content': {'function_name': 'write_final_answers', 'call_id': 'call_9ZykiXtbUDivqi99vtBbOzHS', 'arguments': {'message': "2) The Mona Lisa, painted by Leonardo da Vinci in the early 16th century, is one of the most famous works of art in the world. The painting depicts Lisa Gherardini, a Florentine woman, commissioned by her husband, Francesco del Giocondo. Da Vinci employed techniques such as sfumato, which creates soft transitions between colors, and chiaroscuro, which enhances the depth and volume of the figure, contributing to the painting's lifelike quality. The enigmatic smile of the Mona Lisa has fascinated viewers for centuries, prompting various interpretations about her mood and emotions; it has been the subject of many art theories and psychological studies. The background features a distant landscape with winding paths and a serene river,

    +--------------------------------------------------------------------------------+
    |
    | Student_Agent -> Teacher_Agent: Message type: suggested_function_call
    |
    | {
    |   "function_name": "write_final_answers",
    |   "call_id": 
    | "call_9ZykiXtbUDivqi99vtBbOzHS",
    |   "arguments": {
    |     "message": "2) 
    | The Mona Lisa, painted by Leonardo da Vinci in the early 16th century,
    |  is one of the most famous works of art in the world. The painting 
    | depicts Lisa Gherardini, a Florentine woman, commissioned by her 
    | husband, Francesco del Giocondo. Da Vinci employed techniques such as 
    | sfumato, which creates soft transitions between colors, and 
    | chiaroscuro, which enhances the depth and volume of the figure, 
    | contributing to the painting's lifelike quality. The enigmatic smile 
    | of the Mona Lisa has fascinated viewers for centuries, prompting 
    | various interpretations about her mood and emotions; it has been the 

KeyError: 'content'